In [1]:
import sympy as smp
from itertools import product

In [2]:
tau = smp.symbols("tau")

rho, phi = smp.symbols("rho phi", cls=smp.Function)
rho = rho(tau)
phi = phi(tau)

In [3]:
x = rho*smp.cos(phi)
y = rho*smp.sin(phi)
z = smp.trigsimp(smp.Rational(1, 2)*(x*x + y*y))

In [4]:
variables = [rho, phi]
n = len(variables)
g = [[0]*n for _ in range(n)]

for i in range(0, n):
    for j in range(0, n):
        for v in [x, y, z]:
            g[i][j] += smp.diff(v, variables[i])*smp.diff(v, variables[j])
        g[i][j] = g[i][j].simplify()

In [5]:
g[0][0]

rho(tau)**2 + 1

In [6]:
g[1][1]

rho(tau)**2

In [7]:
L = smp.sympify(0)

for i in range(0, n):
    for j in range(0, n):
        L += g[i][j]*smp.diff(variables[i], tau)*smp.diff(variables[j], tau)

# Estas tres lineas toman algo de tiempo
eel = smp.euler_equations(L, variables, tau)

In [8]:
Gamma = [[[0 for _ in range(n)] for _ in range(n)] for _ in range(n)] # Matriz 2*2*2

for i in range(0, n): # primer indice
    # Resolvemos d^2/dt^2 (v^i) = elem
    elem = smp.solve(eel[i], smp.diff(variables[i], tau, tau))[0]
    elem = smp.expand(elem)
    # elem será una suma de coeficientes lineares en las derivadas de las variables_tau
    # Por ahora hacemos un loop doble, pero tal vez sería posible extraer las derivadas directamente
    # https://docs.sympy.org/latest/index.html
    for j in range(0, n):
        for k in range(0, n):
            var = smp.diff(variables[j], tau)*smp.diff(variables[k], tau)
            Gamma[i][j][k] = -elem.coeff(var)
            if j != k: # The common pitfall
                Gamma[i][j][k] = Gamma[i][j][k]/2

In [9]:
Gamma[0][0][0]

rho(tau)/(rho(tau)**2 + 1)

In [10]:
Gamma[0][1][1]

-rho(tau)/(rho(tau)**2 + 1)

In [11]:
Gamma[1][0][1]

1/rho(tau)

In [12]:
Gamma[1][1][0]

1/rho(tau)

In [13]:
Riemman = [[[[0 for _ in range(2)] for _ in range(2)] for _ in range(2)] for _ in range(2)] # Matriz 2*2*2*2

for i, j, k, l in product(range(0, 2), repeat=4):
    Riemman[i][j][k][l] = smp.diff(Gamma[i][l][j], variables[k]) - smp.diff(Gamma[i][k][j], variables[l])
    for alpha in range(0, 2):
        Riemman[i][j][k][l] += Gamma[i][k][alpha]*Gamma[alpha][l][j] - Gamma[i][l][alpha]*Gamma[alpha][k][j]
    # https://docs.sympy.org/latest/tutorials/intro-tutorial/simplification.html#trigsimp
    Riemman[i][j][k][l] = smp.simplify(Riemman[i][j][k][l])

In [14]:
Riemman[1][0][1][0]

1/(rho(tau)**2 + 1)

In [15]:
Riemman[1][0][0][1]

-1/(rho(tau)**2 + 1)

In [16]:
Riemman[0][1][1][0]

-rho(tau)**2/(rho(tau)**2 + 1)**2

In [17]:
Riemman[0][1][0][1]

rho(tau)**2/(rho(tau)**2 + 1)**2

In [18]:
Ricci = [[0]*2 for _ in range(n)] # this does not work: [[0]*3]*3

for i, j in product(range(0, n), repeat=2):
    for alpha in range(0, n):
        Ricci[i][j] += Riemman[alpha][i][alpha][j]

In [19]:
Ricci[1][1]

rho(tau)**2/(rho(tau)**2 + 1)**2

In [20]:
R = smp.sympify(0)

for i, j in product(range(0, 2), repeat=2):
    R += (smp.Matrix(g)**(-1)).row(i)[j]*Ricci[i][j]

R

2/(rho(tau)**2 + 1)**2

Lo hacemos más general con una función

In [58]:
import sympy as smp
from itertools import product

def computar(variables_str, g):
    """
    Con la métrica como una matriz de strings, calcular simbolos de christoffel, etc
    """
    tau = smp.symbols("tau")

    u, v = smp.symbols(variables_str, cls=smp.Function)
    u = u(tau)
    v = v(tau)

    variables = [u, v]

    n = len(variables)
    
    for i, j in product(range(0, n), repeat=2):
        g[i][j] = smp.sympify(g[i][j])

    L = smp.sympify(0)

    for i, j in product(range(0, n), repeat=2):
            L += g[i][j]*smp.diff(variables[i], tau)*smp.diff(variables[j], tau)
    
    # Estas tres lineas toman algo de tiempo
    eel = smp.euler_equations(L, variables, tau)

    Gamma = [[[0 for _ in range(n)] for _ in range(n)] for _ in range(n)] # Matriz 2*2*2

    for i in range(0, n): # primer indice
        # Resolvemos d^2/dt^2 (v^i) = elem
        elem = smp.solve(eel[i], smp.diff(variables[i], tau, tau))[0]
        elem = smp.expand(elem)
        # elem será una suma de coeficientes lineares en las derivadas de las variables_tau
        # Por ahora hacemos un loop doble, pero tal vez sería posible extraer las derivadas directamente
        # https://docs.sympy.org/latest/index.html
        for j, k in product(range(0, n), repeat=2):
            var = smp.diff(variables[j], tau)*smp.diff(variables[k], tau)
            Gamma[i][j][k] = -elem.coeff(var)
            if j != k: # The common pitfall
                Gamma[i][j][k] = Gamma[i][j][k]/2
                    
    Riemman = [[[[0 for _ in range(n)] for _ in range(n)] for _ in range(n)] for _ in range(n)] # Matriz 2*2*2*2

    for i, j, k, l in product(range(0, n), repeat=4):
        Riemman[i][j][k][l] = smp.diff(Gamma[i][l][j], variables[k]) - smp.diff(Gamma[i][k][j], variables[l])
        for alpha in range(0, n):
            Riemman[i][j][k][l] += Gamma[i][k][alpha]*Gamma[alpha][l][j] - Gamma[i][l][alpha]*Gamma[alpha][k][j]
        # https://docs.sympy.org/latest/tutorials/intro-tutorial/simplification.html#trigsimp
        Riemman[i][j][k][l] = smp.simplify(Riemman[i][j][k][l])

    Ricci = [[0]*2 for _ in range(n)] # this does not work: [[0]*3]*3

    for i, j in product(range(0, n), repeat=2):
        for alpha in range(0, n):
            Ricci[i][j] += Riemman[alpha][i][alpha][j]

    R = smp.sympify(0)

    for i, j in product(range(0, n), repeat=2):
        R += (smp.Matrix(g)**(-1)).row(i)[j]*Ricci[i][j]

    return Gamma, Riemman, Ricci, R

In [59]:
# Ejercicio 7: Apartados a y b
variables = "chi phi"
g = [["a^2", "0"], ["0", "(a^2)*sinh(chi(tau))^2"]] # (a^2)*

# result = computar(variables, g)

Gamma, Riemman, Ricci, R = computar(variables, g)

In [60]:
Gamma[0][1][1]

-sinh(2*chi(tau))/2

In [61]:
Gamma[1][0][1]

1/tanh(chi(tau))

In [62]:
Gamma[1][1][0]

1/tanh(chi(tau))

In [63]:
Riemman[1][0][1][0]

-1

In [64]:
smp.trigsimp(smp.expand_trig(Riemman[0][1][0][1]))

-sinh(chi(tau))**2

In [65]:
Ricci[0][0]

-1

In [66]:
Ricci[1][1]

sinh(2*chi(tau))/(2*tanh(chi(tau))) - cosh(2*chi(tau))

In [67]:
R.simplify()

-2/a**2

Tiene escalar de ricci constante e igual a menos el de la esfera, es el hiperboloide. 

In [18]:
# Ejercicio 8: apartado b
variables = "x y"
g = [["0", "1/(x^2 + y^2)"], ["1/(x^2 + y^2)", "0"]]

# No funciona, el programa espera que la ec. de lagrange con respecto a x sea una ec. diferencial en dot dot x
Gamma, Riemman, Ricci, R = computar(variables, g) 

IndexError: list index out of range

In [68]:
# Ejercicio 11: apartado b
variables = "r theta"
g = [["1", "0"], ["0", "r(tau)^2"]]

Gamma, Riemman, Ricci, R = computar(variables, g) 

In [69]:
R # Curvatura cero, estamos en el plano

0

In [70]:
# Ejercicio 13
variables = "theta phi"
g = [
        ["(b + a*sin(phi(tau)))^2", "0"], 
        ["0", "a^2"]
]

Gamma, Riemman, Ricci, R = computar(variables, g) 

In [71]:
Gamma[0][1][0]

a*cos(phi(tau))/(a*sin(phi(tau)) + b)

In [72]:
Gamma[1][0][0].factor()

-(a*sin(phi(tau)) + b)*cos(phi(tau))/a

In [73]:
Riemman[1][0][1][0]

(a*sin(phi(tau)) + b)*sin(phi(tau))/a

In [74]:
Riemman[0][1][0][1]

a*sin(phi(tau))/(a*sin(phi(tau)) + b)

In [75]:
Ricci[0][0]

(a*sin(phi(tau)) + b)*sin(phi(tau))/a

In [76]:
Ricci[1][1]

a*sin(phi(tau))/(a*sin(phi(tau)) + b)

In [77]:
R

2*sin(phi(tau))/(a*(a*sin(phi(tau)) + b))

In [ ]:
import sympy as smp
from itertools import product

def computar(variables_str, g):
    """
    Con la métrica como una matriz de strings, calcular simbolos de christoffel, etc
    Modificado para que funcione en cualquier número de dimensiones
    """
    tau = smp.symbols("tau")

    variables = smp.symbols(variables_str, cls=smp.Function)
    variables = [v(tau) for v in variables]

    n = len(variables)
    
    for i, j in product(range(0, n), repeat=2):
        g[i][j] = smp.sympify(g[i][j])

    L = smp.sympify(0)

    for i, j in product(range(0, n), repeat=2):
        L += g[i][j]*smp.diff(variables[i], tau)*smp.diff(variables[j], tau)
    
    # Estas tres lineas toman algo de tiempo
    eel = smp.euler_equations(L, variables, tau)

    Gamma = [[[0 for _ in range(n)] for _ in range(n)] for _ in range(n)] # Matriz n*n*n

    for i in range(0, n): # primer indice
        # Resolvemos d^2/dt^2 (v^i) = elem
        elem = smp.solve(eel[i], smp.diff(variables[i], tau, tau))[0]
        elem = smp.expand(elem)
        # elem será una suma de coeficientes lineares en las derivadas de las variables_tau
        # Por ahora hacemos un loop doble, pero tal vez sería posible extraer las derivadas directamente
        # https://docs.sympy.org/latest/index.html
        for j, k in product(range(0, n), repeat=2):
            var = smp.diff(variables[j], tau)*smp.diff(variables[k], tau)
            Gamma[i][j][k] = -elem.coeff(var)
            if j != k: # The common pitfall
                Gamma[i][j][k] = Gamma[i][j][k]/2
                    
    Riemman = [[[[0 for _ in range(n)] for _ in range(n)] for _ in range(n)] for _ in range(n)] # Matriz n*n*n*n

    for i, j, k, l in product(range(0, n), repeat=4):
        Riemman[i][j][k][l] = smp.diff(Gamma[i][l][j], variables[k]) - smp.diff(Gamma[i][k][j], variables[l])
        for alpha in range(0, n):
            Riemman[i][j][k][l] += Gamma[i][k][alpha]*Gamma[alpha][l][j] - Gamma[i][l][alpha]*Gamma[alpha][k][j]
        # https://docs.sympy.org/latest/tutorials/intro-tutorial/simplification.html#trigsimp
        # Riemman[i][j][k][l] = smp.simplify(Riemman[i][j][k][l])
        Riemman[i][j][k][l] = smp.expand_trig(Riemman[i][j][k][l])
        Riemman[i][j][k][l] = smp.trigsimp(Riemman[i][j][k][l])

    Ricci = [[0 for _ in range(n)] for _ in range(n)] # Matriz n*n

    for i, j in product(range(0, n), repeat=2):
        for alpha in range(0, n):
            Ricci[i][j] += Riemman[alpha][i][alpha][j]

    R = smp.sympify(0)

    # g_inversa = inverso_analitico(g)

    for i, j in product(range(0, n), repeat=2):
        R += (smp.Matrix(g)**(-1)).row(i)[j]*Ricci[i][j]
        # R += g_inversa[i][j]*Ricci[i][j]
    R = smp.expand_trig(R)
    R = smp.trigsimp(R)
    return Gamma, Riemman, Ricci, R

def calcular_metrica(variables_str, cambios_str, as_strings=True):
    """
    Calcula la métrica de un cambio de coordenadas aleatorio en el espacio plano. Necesita las nuevas coordenadas y las 
    viejas coordenadas en función de las nuevas. 
    Ejemplo: pasar de espacio plano con (x, y, z) a espacio plano con (r, theta, phi) y una nueva métrica se hace con:
    calcular_metrica(["r", "theta", "phi"], ["r*sin(theta)*cos(phi)", "r*sin(theta)*sin(phi)", "r*cos(theta)"])
    """
    n = len(cambios_str)
    g = [[0 for _ in range(0, n)] for _ in range(0, n)] # Matriz n*n

    cambio = [smp.sympify(v) for v in cambios_str]
    variables = smp.symbols(variables_str)

    for i, j in product(range(0, n), repeat=2):
        for v in cambio:
            # Implicitamente g_(ij) de la que venimos es la delta de kroneker
            g[i][j] += smp.diff(v, variables[i])*smp.diff(v, variables[j])
        g[i][j] = g[i][j].simplify()

    # Cambiamos las variables por variables que dependen explicitamente de tau    
    tau = smp.symbols("tau")
    variables_tau = smp.symbols(variables_str, cls=smp.Function)
    variables_tau = [v(tau) for v in variables_tau]

    for i, j in product(range(0, n), repeat=2):
        for k in range(0, n):
            g[i][j] = g[i][j].subs(variables[k], variables_tau[k])

    if as_strings:
        return [[str(g[i][j]) for j in range(0, n)] for i in range(0, n)]
    
    return g

kroneker = lambda i, j: 1 if i == j else 0

def inverso_analitico(M): 
    """
    Calcula el inverso de la matriz M de forma analítica con la descomposición de Cayley-Hamilton
    https://en.wikipedia.org/wiki/Invertible_matrix#Analytic_solution
    Sólo implementado hasta orden 3
    """

    m = len(M)
    M_inv = [[0 for _ in range(0, m)] for _ in range(0, m)]

    smp_M = smp.Matrix(M)

    traza, determinante = smp.trace(smp_M), smp.Determinant(smp_M).doit()

    if determinante == 0: raise ValueError("La matriz dada no tiene inverso")

    if m == 2:
        for i, j in product(range(0, m), repeat=2):
            M_inv[i][j] = (1/determinante)*(traza*kroneker(i, j) - M[i][j])
        return M_inv
    if m == 3:
        traza_cuad = smp.trace(smp_M**2)
        for i, j in product(range(0, m), repeat=2):
            M_inv[i][j] = (1/determinante)*((1/2)*(traza**2 - traza_cuad**2)*kroneker(i, j) - M[i][j]*traza + (smp_M**2).row(i)[j])
        return M_inv
    
def computar_e_imprimir(variables_str:list, g):
    n = len(variables_str)

    Gamma, Riemman, Ricci, R = computar(variables_str, g)
        
    v = variables_str

    for i, j, k in product(range(0, n), repeat=3):
        if Gamma[i][j][k] != 0:
            s = str(Gamma[i][j][k]).replace("(tau)", "")
            print(f"Gamma^{v[i]}_({v[j]} {v[k]}) = {s}")
    print()
    for i, j, k, l in product(range(0, n), repeat=4):
        if Riemman[i][j][k][l] != 0:
            s = str(Riemman[i][j][k][l]).replace("(tau)", "")
            print(f"Riemman^{v[i]}_({v[j]} {v[k]} {v[l]}) = {s}")
    print()
    for i, j in product(range(0, n), repeat=2):
        if Ricci[i][j] != 0:
            s = str(Ricci[i][j]).replace("(tau)", "")
            print(f"Ricci_({v[i]} {v[j]}) = {s}")
    print()
    print("R = ", str(R).replace("(tau)", ""))

In [4]:
# Dos esfera
variables = ["r", "theta", "phi"]

g = [
        ["1", "0", "0"],
        ["0", "r^2", "0"],
        ["0", "0", "r^2*sin(theta(tau))^2"]
]

variables = variables[1:]
g = [row[1:] for row in g][1:]

computar_e_imprimir(variables, g)

Gamma^theta_(phi phi) = -sin(2*theta)/2
Gamma^phi_(theta phi) = 1/tan(theta)
Gamma^phi_(phi theta) = 1/tan(theta)

Riemman^theta_(phi theta phi) = sin(theta)**2
Riemman^theta_(phi phi theta) = -sin(theta)**2
Riemman^phi_(theta theta phi) = -1
Riemman^phi_(theta phi theta) = 1

Ricci_(theta theta) = 1
Ricci_(phi phi) = sin(theta)**2

2/r**2


In [5]:
# Tres esfera
v = ["r", "theta", "varphi", "phi"]
old_v = ["r*cos(theta)", "r*sin(theta)*cos(varphi)", "r*sin(theta)*sin(varphi)*cos(phi)", "r*sin(theta)*sin(varphi)*sin(phi)"]

g = calcular_metrica(v, old_v)

v = v[1:]
g = [row[1:] for row in g][1:]

computar_e_imprimir(v, g)

Gamma^theta_(varphi varphi) = -sin(theta)*cos(theta)
Gamma^theta_(phi phi) = -sin(theta)*sin(varphi)**2*cos(theta)
Gamma^varphi_(theta varphi) = 1/tan(theta)
Gamma^varphi_(varphi theta) = 1/tan(theta)
Gamma^varphi_(phi phi) = -sin(2*varphi)/2
Gamma^phi_(theta phi) = 1/tan(theta)
Gamma^phi_(varphi phi) = 1/tan(varphi)
Gamma^phi_(phi theta) = 1/tan(theta)
Gamma^phi_(phi varphi) = 1/tan(varphi)

Riemman^theta_(varphi theta varphi) = sin(theta)**2
Riemman^theta_(varphi varphi theta) = -sin(theta)**2
Riemman^theta_(phi theta phi) = sin(theta)**2*sin(varphi)**2
Riemman^theta_(phi phi theta) = -sin(theta)**2*sin(varphi)**2
Riemman^varphi_(theta theta varphi) = -1
Riemman^varphi_(theta varphi theta) = 1
Riemman^varphi_(phi varphi phi) = sin(theta)**2*sin(varphi)**2
Riemman^varphi_(phi phi varphi) = -sin(theta)**2*sin(varphi)**2
Riemman^phi_(theta theta phi) = -1
Riemman^phi_(theta phi theta) = 1
Riemman^phi_(varphi varphi phi) = -sin(theta)**2
Riemman^phi_(varphi phi varphi) = sin(theta)**2

R

In [27]:
v = ["r", "theta", "phi"]
old_v = ["(b + a*sin(phi))*cos(theta)", "(b + a*sin(phi))*sin(theta)", "a*cos(phi)"]

g = calcular_metrica(v, old_v)

v = v[1:]
g = [row[1:] for row in g][1:]

computar_e_imprimir(v, g)

Gamma^theta_(theta phi) = a*cos(phi)/(a*sin(phi) + b)
Gamma^theta_(phi theta) = a*cos(phi)/(a*sin(phi) + b)
Gamma^phi_(theta theta) = -sin(phi)*cos(phi) - b*cos(phi)/a

Riemman^theta_(phi theta phi) = a*sin(phi)/(a*sin(phi) + b)
Riemman^theta_(phi phi theta) = -a*sin(phi)/(a*sin(phi) + b)
Riemman^phi_(theta theta phi) = -(sin(phi) + b/a)*sin(phi)
Riemman^phi_(theta phi theta) = (sin(phi) + b/a)*sin(phi)

Ricci_(theta theta) = (sin(phi) + b/a)*sin(phi)
Ricci_(phi phi) = a*sin(phi)/(a*sin(phi) + b)

2*sin(phi(tau))/(a*(a*sin(phi(tau)) + b))


In [6]:
#Cambio de phi -> phi + pi/2, Cambia sinphi->cosphi, cosphi -> -sinphi
v = ["r", "theta", "phi"]
old_v = ["(b + a*cos(phi))*cos(theta)", "(b + a*cos(phi))*sin(theta)", "-a*sin(phi)"]

calcular_metrica(v, old_v)

[['0', '0', '0'], ['0', '(a*cos(phi(tau)) + b)**2', '0'], ['0', '0', 'a**2']]

In [7]:
# Ejercicio 8: apartado b
variables = "x y"
g = [["0", "1/(x^2 + y^2)"], ["1/(x^2 + y^2)", "0"]]

# No funciona, el programa espera que la ec. de lagrange con respecto a x sea una ec. diferencial en dot dot x
Gamma, Riemman, Ricci, R = computar(variables, g) 

IndexError: list index out of range

In [10]:
# Esta superficie 
v = ["d", "v", "u"] # d es una variable muda
old_v = ["(4 + sin(2*v)*sin(2*u))*sin(3*v)", "sin(2*v)*cos(2*u) + 8*v - 4" ,"(4 + sin(2*v)*sin(2*u))*cos(3*v)"]

g = calcular_metrica(v, old_v)
print(g)

# Reducimos la métrica de esta manera para calcular las propiedades de la superficie intrinsicamente
v = v[1:]
g = [g[i][1:] for i in [1, 2]]
print(g)

Gamma, Riemman, Ricci, R = computar(v, g) # se queda atascado encontrando la inversa de la matriz
# https://en.wikipedia.org/wiki/Invertible_matrix#Analytic_solution

[['0', '0', '0'], ['0', '4*(cos(2*u)*cos(2*v) + 4)**2 + (48*sin(3*v) - 5*sin(2*u - 5*v) + sin(2*u - v) + sin(2*u + v) - 5*sin(2*u + 5*v))**2/16 + (48*cos(3*v) + 5*cos(2*u - 5*v) - cos(2*u - v) + cos(2*u + v) - 5*cos(2*u + 5*v))**2/16', '-16*sin(2*u)*sin(2*v)'], ['0', '-16*sin(2*u)*sin(2*v)', '4*sin(2*v)**2']]
[['4*(cos(2*u)*cos(2*v) + 4)**2 + (48*sin(3*v) - 5*sin(2*u - 5*v) + sin(2*u - v) + sin(2*u + v) - 5*sin(2*u + 5*v))**2/16 + (48*cos(3*v) + 5*cos(2*u - 5*v) - cos(2*u - v) + cos(2*u + v) - 5*cos(2*u + 5*v))**2/16', '-16*sin(2*u)*sin(2*v)'], ['-16*sin(2*u)*sin(2*v)', '4*sin(2*v)**2']]


In [9]:
g

[[4*(cos(2*u)*cos(2*v) + 4)**2 + (48*sin(3*v) - 5*sin(2*u - 5*v) + sin(2*u - v) + sin(2*u + v) - 5*sin(2*u + 5*v))**2/16 + (48*cos(3*v) + 5*cos(2*u - 5*v) - cos(2*u - v) + cos(2*u + v) - 5*cos(2*u + 5*v))**2/16,
  -16*sin(2*u)*sin(2*v)],
 [-16*sin(2*u)*sin(2*v), 4*sin(2*v)**2]]